In [ ]:
import cv2
from sklearn.cluster import KMeans
from skimage.color import label2rgb
import matplotlib.pyplot as plt
import numpy as np
import scipy.io as sio

plt.ioff()
%matplotlib inline
plt.rcParams['figure.figsize'] = [30, 100]

image = cv2.imread("101027.jpg")[:,:,::-1]

In [ ]:
def applyfilter(A, g, disable_blur = False, disable_spatial = False, disable_norm = False, disable_color = False, save_bank_results = False):
    if A.shape[2]==3:
        Agray = cv2.cvtColor(A, cv2.COLOR_RGB2GRAY)
    elif A.shape[2] == 1:
        Agray = A
        
    numRows = Agray.shape[0]
    numCols = Agray.shape[1]
    sizeFactors = np.array([0.25])
    sizeFactors = np.power(2, sizeFactors)
    gabormag = np.ndarray((Agray.shape[0], Agray.shape[1], g.shape[2]*len(sizeFactors)))
    sigmas = np.ndarray((g.shape[2]+1)*len(sizeFactors))
    
#    for i in range(g.shape[2]):
#        plt.subplot(15, 15, i+1)
#        plt.imshow(g[:,:,i], cmap="gray")
#        plt.imshow(cv2.resize(g[:,:,i], (g[:,:,i].shape[0]*sizeFactors[-1], g[:,:,i].shape[1]*sizeFactors[-1]), interpolation = cv2.INTER_LANCZOS4))
#    plt.show()
    
    for i in range(g.shape[2]):
        for s in range(len(sizeFactors)):
            gabormag[:, :, i*len(sizeFactors)+s] = cv2.filter2D(Agray, -1, cv2.resize(g[:,:,i], (int(g[:,:,i].shape[0]*sizeFactors[s]), int(g[:,:,i].shape[1]*sizeFactors[s])), interpolation = cv2.INTER_LANCZOS4), borderType=cv2.BORDER_REPLICATE)
            sigmas[i*len(sizeFactors)+s] = np.sqrt(2)*g.shape[0]*sizeFactors[s]/49

    if save_bank_results == True:
        for i in range(gabormag.shape[2]):
         #   plt.subplot(1, 1, i+1)
            plt.imshow(gabormag[:, :, i])
            plt.axis('off')
            plt.savefig("GEN_tsugf_filterbank_101027_"+str(i+1)+".png", bbox_inches="tight", pad_inches = 0)
         #   plt.show()     
    if disable_blur == False:    
        for i in range(gabormag.shape[2]):
            gabormag[:, :, i] = cv2.GaussianBlur(gabormag[:, :, i], (0, 0), 3*sigmas[i])

    X = np.arange(1, numCols + 1)
    Y = np.arange(1, numRows + 1)
    X, Y = np.meshgrid(X, Y)
        
    numPoints = numRows * numCols
    featureSet = gabormag
    
    if disable_spatial == False:
        featureSet = np.concatenate((featureSet, np.expand_dims(X, axis=2)), 2)
        featureSet = np.concatenate((featureSet, np.expand_dims(Y, axis=2)), 2)
    
    if A.shape[2]==3 and disable_color == False:
        featureSet = np.concatenate((featureSet, np.expand_dims(A[:,:,0], axis=2)), 2)
        featureSet = np.concatenate((featureSet, np.expand_dims(A[:,:,1], axis=2)), 2)
        featureSet = np.concatenate((featureSet, np.expand_dims(A[:,:,2], axis=2)), 2)

    X = featureSet.reshape(numPoints, -1)

    #for i in range(X.shape[1]):
    #    plt.subplot(20, 4, i+1)
    #    plt.imshow(X[:, i].reshape((A.shape[0],A.shape[1])))
    #plt.show()
    
    if disable_norm == False:
        X = X - X.mean(axis=0)
        X = X / X.std(axis=0, ddof=1)
    X = X[:, ~np.isnan(X).any(axis=0)]
    X = X[:, ~np.isinf(X).any(axis=0)]
    
    if disable_color == False:
        X[:,-1:-3] = X[:,-1:-3]/3
    
    L = KMeans(n_clusters=5, n_init=20, max_iter=1000, n_jobs=10).fit(X).labels_
    
    return L

mat = sio.loadmat("../../code/filterbanks/filterbanks.mat")
g = mat["TSUGFfilters"]

#tags = Parallel(n_jobs=5, backend="threading", verbose=10)(delayed(applyfilter)(A, g) for A in images)
#tags = applyfilter(image, g)

plt.imshow(label2rgb(applyfilter(image, g, disable_blur = True, disable_spatial = True, disable_norm = True, disable_color = True).reshape((image.shape[0],image.shape[1]))))
plt.axis('off')
plt.savefig("GEN_tsugf_filterbank_101027_raw"+".png", bbox_inches="tight", pad_inches = 0)
plt.show()

plt.imshow(label2rgb(applyfilter(image, g, disable_spatial = True, disable_norm = True, disable_color = True).reshape((image.shape[0],image.shape[1]))))
plt.axis('off')
plt.savefig("GEN_tsugf_filterbank_101027_blur"+".png", bbox_inches="tight", pad_inches = 0)
plt.show()

#plt.imshow(label2rgb(applyfilter(image, g, disable_color = True, disable_spatial = True).reshape((image.shape[0],image.shape[1]))))
#plt.axis('off')
#plt.savefig("GEN_tsugf_filterbank_101027_blur_norm"+".png", bbox_inches="tight", pad_inches = 0)
#plt.show()

plt.imshow(label2rgb(applyfilter(image, g, disable_color = True).reshape((image.shape[0],image.shape[1]))))
plt.axis('off')
plt.savefig("GEN_tsugf_filterbank_101027_blur_norm_spatial"+".png", bbox_inches="tight", pad_inches = 0)
plt.show()

plt.imshow(label2rgb(applyfilter(image, g, save_bank_results = True).reshape((image.shape[0],image.shape[1]))))
plt.axis('off')
plt.savefig("GEN_tsugf_filterbank_101027_blur_norm_spatial_color"+".png", bbox_inches="tight", pad_inches = 0)
plt.show()
